In [133]:
import pymongo
myclient=pymongo.MongoClient("mongodb://localhost:27017/")
mydb=myclient["Student"]                    #creating database
mycol = mydb["mycollections"]               #creating collections
alldb=myclient.list_database_names()
print(alldb)

['Student', 'Telephone_Directory', 'admin', 'config', 'local']


In [134]:
import ast
#loading file from json
f=open('students.json','r')
#data=[]
for i in f:
    #data.append(i.rstrip())
    #print(i)   
    mycol.insert_one(ast.literal_eval(i))    
    #since each dictionary row is as a string converting to dict and inserting documents one by one
#for doc in mycol.find():    to view available data inserted in document
    #print(doc)    

1) Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [135]:
c=mycol.aggregate([
{"$project":{"_id":0,"name":1,"max_marks":{"$sum":"$scores.score"}}},
{"$sort":{"max_marks":-1}},{"$limit":1}
])
print("student name who scored maximum scores in all (exam, quiz and homework)")
for i in c:
    print(i["name"])
    #print(i)

student name who scored maximum scores in all (exam, quiz and homework)
Jessika Dagenais


2) Find students who scored below average in the exam and pass mark is 40%?

In [136]:
sum=0
count=0
for doc in mycol.find({},{"_id":0,"scores":1}):
    #print(doc["scores"][0]["score"])
    sum+=(doc["scores"][0]["score"])
    count+=1
#print(sum)
#print(count)
avg=sum/count
#print("average =",avg)
print("students who scored below average in the exam and pass mark is 40%")
for doc in mycol.find({},{"_id":0}):
    if doc["scores"][0]["score"]>=40 and doc["scores"][0]["score"]<=avg:
        print(doc["name"])
        #print(doc)

students who scored below average in the exam and pass mark is 40%
Wilburn Spiess
Denisha Cast
Gisela Levin
Tressa Schwing
Rosana Vales
Tamika Schildgen
Gwyneth Garling
Kayce Kenyon
Terica Brugger
Chad Rahe
Jenise Mcguffie
Cassi Heal
Demarcus Audette
Flora Duell
Quincy Danaher
Alix Sherrill
Laureen Salomone
Gwyneth Garling
Aleida Elsass
Alica Pasley
Malisa Jeanes
Harriett Velarde
Mariela Sherer


3)  Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [137]:
print("below 40 marks subjectwise updated as fail")
c=mycol.aggregate([{"$unwind":"$scores"},
    {"$match":{"scores.score": {"$lt": 40}}},
    {"$project":{"_id":0,"student_name":"$name","type_of_exam_failed":"$scores.type", "result": "Fail"}}])
for i in c:
    print(i)

below 40 marks subjectwise updated as fail
{'student_name': 'aimee Zank', 'type_of_exam_failed': 'exam', 'result': 'Fail'}
{'student_name': 'aimee Zank', 'type_of_exam_failed': 'quiz', 'result': 'Fail'}
{'student_name': 'aimee Zank', 'type_of_exam_failed': 'homework', 'result': 'Fail'}
{'student_name': 'Corliss Zuk', 'type_of_exam_failed': 'quiz', 'result': 'Fail'}
{'student_name': 'Bao Ziglar', 'type_of_exam_failed': 'quiz', 'result': 'Fail'}
{'student_name': 'Zachary Langlais', 'type_of_exam_failed': 'homework', 'result': 'Fail'}
{'student_name': 'Wilburn Spiess', 'type_of_exam_failed': 'quiz', 'result': 'Fail'}
{'student_name': 'Jenette Flanders', 'type_of_exam_failed': 'exam', 'result': 'Fail'}
{'student_name': 'Jenette Flanders', 'type_of_exam_failed': 'quiz', 'result': 'Fail'}
{'student_name': 'Daphne Zheng', 'type_of_exam_failed': 'exam', 'result': 'Fail'}
{'student_name': 'Daphne Zheng', 'type_of_exam_failed': 'quiz', 'result': 'Fail'}
{'student_name': 'Sanda Ryba', 'type_of_ex

In [138]:
print("pass in exam")
for i in mycol.find({"scores.0.score":{"$gte":40}},{"_id":0,"name":1}):
    #print(i)
    print(str(i["name"])+str("result pass in exam"))
print("pass in quiz")
for i in mycol.find({"scores.1.score":{"$gte":40}},{"_id":0,"name":1}):
    #print(i)
    print(str(i["name"])+str("result pass in quiz"))
print("pass in homework")
for i in mycol.find({"scores.2.score":{"$gte":40}},{"_id":0,"name":1}):
    #print(i)
    print(str(i["name"])+str("result pass in homework"))

pass in exam
Aurelia Menendezresult pass in exam
Corliss Zukresult pass in exam
Bao Ziglarresult pass in exam
Zachary Langlaisresult pass in exam
Wilburn Spiessresult pass in exam
Salena Olmosresult pass in exam
Sanda Rybaresult pass in exam
Denisha Castresult pass in exam
Marcus Blohmresult pass in exam
Quincy Danaherresult pass in exam
Jessika Dagenaisresult pass in exam
Tambra Mercureresult pass in exam
Verdell Sowinskiresult pass in exam
Gisela Levinresult pass in exam
Tressa Schwingresult pass in exam
Rosana Valesresult pass in exam
Margart Vitelloresult pass in exam
Tamika Schildgenresult pass in exam
Jesusa Rickenbackerresult pass in exam
Rudolph Domingoresult pass in exam
Gwyneth Garlingresult pass in exam
Gwen Honigresult pass in exam
Sadie Jerniganresult pass in exam
Eugene Magdalenoresult pass in exam
Meagan Oakesresult pass in exam
Rachell Amanresult pass in exam
Kayce Kenyonresult pass in exam
Houston Valentiresult pass in exam
Terica Bruggerresult pass in exam
Kurtis Jile

4) Find the total and average of the exam, quiz and homework and store them in a separate collection.

In [139]:
c=mycol.aggregate([
    {"$project":{"_id":0,"name":1,"total":{"$sum":"$scores.score"},"average":{"$avg":"$scores.score"}}}
])
mycol_total_avg=mydb["total_avg_collections"]
for i in c:
    print(i)
    mycol_total_avg.insert_one(i)
#for doc in mycol_total_avg.find({},{"_id":0}):      for viewing inserted elements
    #print(doc)

{'name': 'aimee Zank', 'total': 49.11994783171814, 'average': 16.373315943906046}
{'name': 'Aurelia Menendez', 'total': 184.61969202100374, 'average': 61.53989734033458}
{'name': 'Corliss Zuk', 'total': 139.61606947126907, 'average': 46.53868982375636}
{'name': 'Bao Ziglar', 'total': 138.70712252233457, 'average': 46.23570750744486}
{'name': 'Zachary Langlais', 'total': 203.3963625302728, 'average': 67.79878751009093}
{'name': 'Wilburn Spiess', 'total': 134.0186975547794, 'average': 44.67289918492647}
{'name': 'Jenette Flanders', 'total': 147.2203575476617, 'average': 49.07345251588723}
{'name': 'Salena Olmos', 'total': 229.3959334774732, 'average': 76.46531115915774}
{'name': 'Daphne Zheng', 'total': 112.71677331754347, 'average': 37.57225777251449}
{'name': 'Sanda Ryba', 'total': 220.08328118626565, 'average': 73.36109372875522}
{'name': 'Denisha Cast', 'total': 199.8843572885121, 'average': 66.62811909617069}
{'name': 'Marcus Blohm', 'total': 248.5091538691126, 'average': 82.8363846

5)  Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [140]:
c=mycol_total_avg.aggregate([
    {"$group":{"_id":0,"average_of_total":{"$avg":"$total"}}}
])
for i in c:
    print(i)
average_of_total=i["average_of_total"]
print("scored below average and above 40% in all the categories")
mycol_1=mydb["score_below_avg_above_40_in_all_category"]
for j in mycol_total_avg.find({"total":{"$gte":40,"$lt":average_of_total}},{"_id":0,"name":1}):
    print(j)
    mycol_1.insert_one(j)

{'_id': 0, 'average_of_total': 165.4890901604158}
scored below average and above 40% in all the categories
{'name': 'aimee Zank'}
{'name': 'Corliss Zuk'}
{'name': 'Bao Ziglar'}
{'name': 'Wilburn Spiess'}
{'name': 'Jenette Flanders'}
{'name': 'Daphne Zheng'}
{'name': 'Alix Sherrill'}
{'name': 'Tambra Mercure'}
{'name': 'Dodie Staller'}
{'name': 'Gisela Levin'}
{'name': 'Jesusa Rickenbacker'}
{'name': 'Rudolph Domingo'}
{'name': 'Jonie Raby'}
{'name': 'Edgar Sarkis'}
{'name': 'Laureen Salomone'}
{'name': 'Gwyneth Garling'}
{'name': 'Kaila Deibler'}
{'name': 'Tandra Meadows'}
{'name': 'Sadie Jernigan'}
{'name': 'Carli Belvins'}
{'name': 'Synthia Labelle'}
{'name': 'Mariette Batdorf'}
{'name': 'Ernestine Macfarland'}
{'name': 'Houston Valenti'}
{'name': 'Terica Brugger'}
{'name': 'Lady Lefevers'}
{'name': 'Kurtis Jiles'}
{'name': 'Malisa Jeanes'}
{'name': 'Danika Loeffler'}
{'name': 'Chad Rahe'}
{'name': 'Joaquina Arbuckle'}
{'name': 'Vinnie Auerbach'}
{'name': 'Vina Matsunaga'}
{'name': '

{'name': 'Ernestine Macfarland'}
{'name': 'Barbera Lippman'}
{'name': 'Dinah Sauve'}
{'name': 'Elizabet Kleine'}
{'name': 'Joel Rueter'}
{'name': 'Vinnie Auerbach'}
{'name': 'Grady Zemke'}
{'name': 'Vina Matsunaga'}
{'name': 'Kim Xu'}
{'name': 'Merissa Mann'}
{'name': 'Kam Senters'}
{'name': 'Leonida Lafond'}
{'name': 'Devorah Smartt'}
{'name': 'Mariela Sherer'}
{'name': 'Echo Pippins'}
{'name': 'Linnie Weigel'}
{'name': 'Santiago Dollins'}
{'name': 'Tonisha Games'}
{'name': 'Timothy Harrod'}
{'name': 'Rae Kohout'}


6) Create a new collection which consists of students who scored below the fail mark in all the categories.

In [141]:
mycol_fail_all=mydb["fail_all_subj"]
print("students who scored below the fail mark in all the categories")
for i in mycol.find({"$and":[{"scores.0.score":{"$lt":40}},
            {"scores.1.score":{"$lt":40}},
            {"scores.2.score":{"$lt":40}}]
             },{"_id":0}):
    mycol_fail_all.insert_one(i)
    #print(i["name"])
    print(i)

students who scored below the fail mark in all the categories
{'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}], '_id': ObjectId('63b04592e1835866f7f0f70d')}


7) Create a new collection which consists of students who scored above pass mark in all the categories.

In [142]:
mycol_pass_all=mydb["pass_all_subj"]
print("students who scored above pass mark in all the categories")
for i in mycol.find({"$and":[{"scores.0.score":{"$gte":40}},
            {"scores.1.score":{"$gte":40}},
            {"scores.2.score":{"$gte":40}}]
             },{"_id":0}):
    mycol_pass_all.insert_one(i)
    print(i)

students who scored above pass mark in all the categories
{'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}], '_id': ObjectId('63b04594e1835866f7f0f70e')}
{'name': 'Salena Olmos', 'scores': [{'score': 90.37826509157176, 'type': 'exam'}, {'score': 42.48780666956811, 'type': 'quiz'}, {'score': 96.52986171633331, 'type': 'homework'}], '_id': ObjectId('63b04594e1835866f7f0f70f')}
{'name': 'Denisha Cast', 'scores': [{'score': 45.61876862259409, 'type': 'exam'}, {'score': 98.35723209418343, 'type': 'quiz'}, {'score': 55.90835657173456, 'type': 'homework'}], '_id': ObjectId('63b04594e1835866f7f0f710')}
{'name': 'Marcus Blohm', 'scores': [{'score': 78.42617835651868, 'type': 'exam'}, {'score': 82.58372817930675, 'type': 'quiz'}, {'score': 87.49924733328717, 'type': 'homework'}], '_id': ObjectId('63b04594e1835866f7f0f711')}
{'name': 'Quincy Danaher', 'scores': [{'sc